----

# **Item Based Collaborative Filtering With Memory**

## **Author**   :  **Muhammad Adil Naeem**

## **Contact**   :   **madilnaeem0@gmail.com**
<br>

----



### **Import Libraries**

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import  train_test_split
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

### **Define Dataset**

In [8]:
dataset = {
    'user1': {'Special Ops': 5, 'Criminal Justice': 3, 'Panchayat': 3, 'Sacred Games': 3, 'Apharan': 2, 'Mirzapur': 3},
    'user2': {'Special Ops': 1, 'Criminal Justice': 3, 'Sacred Games': 5, 'Panchayat': 5, 'Mirzapur': 1, 'Apharan': 3},
    'user3': {'Special Ops': 2, 'Panchayat': 5, 'Sacred Games': 3, 'Mirzapur': 4},
    'user4': {'Panchayat': 5, 'Mirzapur': 5, 'Sacred Games': 4},
    'user5': {'Special Ops': 4, 'Criminal Justice': 4, 'Panchayat': 4, 'Mirzapur': 4,'Apharan': 3},
    'user6': {'Special Ops': 4, 'Panchayat': 3, 'Mirzapur': 4, 'Sacred Games': 5, 'Apharan': 3},
    'user7': {'Panchayat': 4, 'Sacred Games': 4, 'Apharan': 1},
}


### **Convert it Into DataFrame**

In [9]:
dataset_df = pd.DataFrame(dataset)
dataset_df.fillna('Not Seen Yet', inplace=True)
dataset_df

,user1,user2,user3,user4,user5,user6,user7
Special Ops,5,1,2.0,Not Seen Yet,4.0,4.0,Not Seen Yet
Criminal Justice,3,3,Not Seen Yet,Not Seen Yet,4.0,Not Seen Yet,Not Seen Yet
Panchayat,3,5,5.0,5.0,4.0,3.0,4.0
Sacred Games,3,5,3.0,4.0,Not Seen Yet,5.0,4.0
Apharan,2,3,Not Seen Yet,Not Seen Yet,3.0,3.0,1.0
Mirzapur,3,1,4.0,5.0,4.0,4.0,Not Seen Yet


### **Extracting Unique Items from a Dataset**

- This function, `unique_items`, generates a list of unique items from a dataset structured as a dictionary. It iterates through each person in the dataset and appends their items to a list. After collecting all items, it converts the list to a set to remove duplicates and then returns it as a list of unique items.

In [10]:
def unique_items():
  unique_items_list = []
  for person in dataset.keys():
    for item in dataset[person]:
      unique_items_list.append(item)
  s = set(unique_items_list)
  unique_items_list = list(s)
  return unique_items_list

unique_items()

['Sacred Games',
 'Special Ops',
 'Apharan',
 'Criminal Justice',
 'Panchayat',
 'Mirzapur']

### **Calculating Item Similarity**

This function, `item_similarity`, computes the cosine similarity between two items based on user ratings in a dataset.

1. It first identifies users who have rated both items and collects their ratings.
2. If there are no common ratings, it returns a similarity of 0.
3. It then extracts the ratings for both items from the common users and computes their cosine similarity, returning the resulting similarity score.

This function can be used to assess how similar two items are based on user preferences.

In [11]:
def item_similarity(item1, item2):
  both_rated = {}
  for person in dataset.keys():
    if item1 in dataset[person] and item2 in dataset[person]:
      both_rated[person] = [dataset[person][item1], dataset[person][item2]]

  number_of_ratings = len(both_rated)
  if number_of_ratings == 0:
    return 0

  item1_ratings = [[dataset[k][item1] for k, v in both_rated.items() if item1 in dataset[k] and item2 in dataset[k]]]
  item2_ratings = [[dataset[k][item2] for k, v in both_rated.items() if item1 in dataset[k] and item2 in dataset[k]]]
  cs = cosine_similarity(item1_ratings, item2_ratings)
  return cs[0][0]

In [13]:
print('Cosine Similarity: ', item_similarity( 'Panchayat','Special Ops'))

Cosine Similarity:  0.8036972934368554


### **Finding Most Similar Items**

This function, `most_similer_items`, identifies items similar to a given target item.

1. It retrieves a list of unique items using the `unique_items` function.
2. It calculates the similarity score between the target item and each other item in the list, excluding the target item itself.
3. The scores are stored as tuples of similarity and the item pair.
4. Finally, it sorts the scores in descending order and returns the sorted list of item similarity pairs.

This function can be used to recommend items based on their similarity to a specified item.

In [16]:
def most_similer_items(target_item):
  un_list = unique_items()
  scores = [(item_similarity(target_item, other_item), target_item + "--->" + other_item) for other_item in un_list if other_item != target_item]
  scores.sort(reverse=True)
  return scores

In [17]:
print(most_similer_items("Panchayat"))

[(0.9701425001453318, 'Panchayat--->Criminal Justice'), (0.9578262852211513, 'Panchayat--->Sacred Games'), (0.9389710680668848, 'Panchayat--->Apharan'), (0.9146753125449716, 'Panchayat--->Mirzapur'), (0.8036972934368554, 'Panchayat--->Special Ops')]


### **Generating Recommended Series for a User**

This function, `target_series_for_user`, provides a list of recommended items for a specified user based on their current ratings.

1. It initializes a list to store items rated by the target user.
2. It retrieves a list of unique items from the dataset.
3. It appends the user's rated items to the list.
4. It calculates the difference between all unique items and the user's rated items to identify recommendations.
5. If no recommendations are available, it returns a message indicating this along with an empty list; otherwise, it returns the list of recommended items along with the user's rated items.

This function can be used to suggest new items to a user based on what they haven't rated yet.

In [21]:
def target_series_for_user(target_person):
  target_person_series_list = []
  unique_list = unique_items()
  for item in dataset[target_person]:
    target_person_series_list.append(item)

  s = set(unique_list)
  unique_list = list(s)
  recomended_series = list(s.difference(target_person_series_list))
  a = len(recomended_series)
  if a==0:
    return "No Recomended Series", [] # Return an empty list as the second value
  return recomended_series,target_person_series_list

# **Creating a DataFrame of Unseen and Seen Series**

This code calls the `target_series_for_user` function with the user identifier 'user7' to obtain two lists: `unseen_series` (items not rated by the user) and `seen_series` (items rated by the user).

It then creates a dictionary with these two lists and converts it into a Pandas DataFrame, which organizes the data for easy viewing and analysis. The resulting DataFrame will have two columns: "Unseen Series" and "Seen Series".

In [23]:
unseen_series, seen_series = target_series_for_user('user7')
dict = {'Unseen Series': unseen_series, 'Seen Series': seen_series}
pd.DataFrame(dict)

,Unseen Series,Seen Series
0,Special Ops,Panchayat
1,Mirzapur,Sacred Games
2,Criminal Justice,Apharan
